### Add all imports


In [2]:
# from utils import *
import re
from pyarabic.araby import strip_diacritics
import numpy as np
import unicodedata
import nltk
import torch
from torch import lstm_cell, nn
import time
import random
import pickle as pkl
import tensorflow as tf
import pickle

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Dense, Dropout, LSTM, Bidirectional, TimeDistributed , Input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import glorot_normal
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split


In [3]:

########################################################################################
# Read the letters from the pickle files which we will use
def get_letters():
    file_path = 'constants/arabic_letters.pickle'
    with open(file_path, 'rb') as file:
        letters = pickle.load(file)
    letters.add("<s>")
    letters.add("</s>")
    letters.add("<PAD>")
    return letters
########################################################################################
# Read the diacritics from the pickle files which we will use
def get_diacritics():
    file_path = 'constants/diacritics.pickle'
    with open(file_path, 'rb') as file:
        diacritics = pickle.load(file)
    # diacritics.add("<s>")
    # diacritics.add("</s>")
    # diacritics.add("<PAD>")
    return diacritics
########################################################################################
# Read the diacritics from the pickle files which we will use
def get_diacritics2id():

    file_path = 'constants/diacritic2id.pickle'
    with open(file_path, 'rb') as file:
        diacritics2id = pickle.load(file)
    # add no tashkeel
    return diacritics2id

########################################################################################
# Read TRAINING dataset given
def read_training_dataset(file_path = "dataset/train.txt"):
    training_sentences = []
    with open(file_path, "r", encoding="utf-8") as file:
        # Read each line in the file
        for line in file:
            # Strip any leading or trailing whitespace from the line
            line = line.strip()
            # Add the line to the list
            training_sentences.append(line)
    # if(len(training_sentences)==50000):
    #     print("Read training set successfully")
    return training_sentences

########################################################################################
# Read DEV dataset given
def read_dev_dataset(file_path = "dataset/val.txt"):
    dev = []
    with open(file_path, "r", encoding="utf-8") as file:
        # Read each line in the file
        for line in file:
            line = line.strip()
            dev.append(line)
    #print(len(dev))
    # if(len(dev)==2500):
    #     print("Read validation set successfully")
    return dev

########################################################################################

def separate_word_to_letters_diacritics(arabic_text, arabic_letters=get_letters()):
    # Normalize the text to handle different Unicode representations
    normalized_text = unicodedata.normalize('NFKD', arabic_text)
    letters = []
    diacritics = []
    # arabic_text = arabic_text[::-1]
    # for ind in range(len(arabic_text)):
    #     print(arabic_text[ind])
    ind=0

    while ind < len(arabic_text):
        temp=[]
        if not unicodedata.combining(arabic_text[ind]):
            # print(arabic_text[ind])
        # if arabic_text[ind] in arabic_letters:
            letters.append(arabic_text[ind])
            # print("added to letters",arabic_text[ind])

            if(ind+1 < len(arabic_text) and not unicodedata.combining(arabic_text[ind+1])):
                diacritics.append(temp)
                # print("added to diacritics from 1st",temp)
            if(ind == (len(arabic_text)-1)):
              diacritics.append(temp)
            ind+=1

        else:
            while ind < len(arabic_text) and unicodedata.combining(arabic_text[ind]):
                # diacritics.pop(0)
                # print(arabic_text[ind])
                temp.append(arabic_text[ind])
                ind+=1
            temp=unicodedata.normalize('NFC', ''.join(temp))
            # temp=[temp[::-1]]
            diacritics.append([temp])
            # print("added to diacritics",temp)
    # letters.reverse()
    # diacritics.reverse()
    return letters, diacritics

########################################################################################
def tokenize_to_vocab(data, vocab):
    tokenized_sentences_word, tokenized_sentences_letters, tokenized_sentences_diacritics = [], [],[]

    for d in (data):
            tokens = nltk.word_tokenize(d, language="arabic", preserve_line=True)
            # Add the start sentence <s> and end sentence </s> tokens at the beginning and end of each tokenized sentence
            tokens.reverse()
            tokens.insert(0,"<s>")
            tokens.append("</s>")

            vocab.update(tokens)

            word_letters=[]
            word_diacritics=[]
            for token in (tokens):
                if token != "<s>" and token != "</s>":
                    # letters = separate_arabic_to_letters(token)
                    letter, diacritic = separate_word_to_letters_diacritics(token)
                    word_diacritics.append(diacritic)
                    word_letters.append(letter)
                else:
                    word_letters.append(token)
                    word_diacritics.append(token)

            tokenized_sentences_letters.append(word_letters)
            tokenized_sentences_diacritics.append(word_diacritics)
            tokenized_sentences_word.append(tokens)

    return vocab, tokenized_sentences_word,tokenized_sentences_letters,tokenized_sentences_diacritics

def extract_sentences(training_dataset):
    # This pattern keeps Arabic letters, diacritics, and whitespaces and endlines
    pattern = re.compile(r'[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\s,.؟،«»؛(){};:!?\-\'"]')

    # Replace unmatched characters with an empty string
    cleaned_corpus = [re.sub(pattern, "", t) for t in training_dataset]
    cleaned_corpus = [re.sub("\s\s+", " ", c) for c in cleaned_corpus]

    print(len(cleaned_corpus))

    data,labels = [],[]

    first = True
    for c in cleaned_corpus:
        sentences = re.split(r'[,.؟،«»؛(){};:!?\-\'"]+', c)  # split on all punctuation
        labels += sentences

        without_dialects = [
            strip_diacritics(s) for s in sentences
        ]  # get the letters without dialects
        data += without_dialects


    # remove any spaces from line
    data = [d.strip() for d in data]
    labels = [l.strip() for l in labels]

    # remove empty lines
    data = [i for i in data if i]
    labels = [i for i in labels if i]
    return data,labels,cleaned_corpus


########################################################################################

In [5]:
#1- Read data embeddings we have for letters and diacritics
letters, diacritics, diacritics2id = get_letters() , get_diacritics(), get_diacritics2id()
#2- Have mapping ready
#### Letters ---- IDs
letters2id = {item: index for index, item in enumerate(letters)}
letters2id[' ']=len(letters2id)-1
id2letters = {index: item for index, item in enumerate(letters)}

# add <PAD> to the mapping
diacritics2id['<PAD>'] = len(diacritics2id)
diacritics2id['<s>'] = len(diacritics2id)
diacritics2id['</s>'] = len(diacritics2id)
id2diacritics = {value: key for key, value in diacritics2id.items()}


for diacritic, id in diacritics2id.items():
        if diacritic not in diacritics:
          diacritics.add(diacritic)

for letter, id in letters2id.items():
        if letter not in letters:
          letters.add(letter)

for id, dic in id2diacritics.items():
      print(id, dic)

0 َ
1 ً
2 ُ
3 ٌ
4 ِ
5 ٍ
6 ْ
7 ّ
8 َّ
9 ًّ
10 ُّ
11 ٌّ
12 ِّ
13 ٍّ
14 
15 <PAD>
16 <s>
17 </s>


### tHE MAIN MODEL AND FEEDING 

In [1]:
# letters, diacritics, diacritics2id
def map_data(data_raw):
    ''' Splists data lines into an array of charachers as integers and an array of discritics as one-hot-encodings '''

    # initialize data and diacritics lists
    X = list()
    Y = list()

    # loop on data lines
    for line in data_raw:

        lit,dicc=separate_word_to_letters_diacritics(line)
        # print(len(lit),len(dicc))
        # for each sub list in dicc , if the len of the list is greater than 1 then take the last element only
        # if the size is zero assert that the size is zero
        # for i in range(len(dicc)):
        #     if len(dicc[i])>=1:
        #         dicc[i]=dicc[i][-1]
        #     elif len(dicc[i])==0:
        #         dicc[i]='<s>'

        # initialize line data and diacritics lists and add start of sentence character
        x = [letters2id['<s>']]
        y = [diacritics2id['<s>']]
        # x=[]
        # y=[]
        for i in range(min(len(lit),len(dicc))):
            if len(dicc[i])>=1:
                dicc[i]=dicc[i][0]
            elif len(dicc[i])==0:
                dicc[i]=''
            if dicc[i] not in diacritics2id:
                dicc[i]=''

            if lit[i] not in letters2id:
                lit[i]='<PAD>'
                dicc[i]='<PAD>'

            x.append(letters2id[lit[i]])
            y.append(diacritics2id[dicc[i]])

        # assert characters list length equals diacritics list length
        # assert(len(x) == len(y))

        # append end of sentence character
        x.append(letters2id['</s>'])
        y.append(diacritics2id['</s>'])

        # convert diacritics integers to one_hot_encodings
        # print(y)
        y = to_categorical(y, len(diacritics2id))

        # append line's data and diacritics lists to total data and diacritics lists
        X.append(x)
        Y.append(y)
    # convert lists to numpy arrays
    # X = np.asarray(X)
    # Y = np.asarray(Y)

    return X, Y


def map_data(data_raw):
    ''' Splists data lines into an array of charachers as integers and an array of discritics as one-hot-encodings '''

    # initialize data and diacritics lists
    X = list()
    Y = list()

    # loop on data lines
    for line in data_raw:

        lit,dicc=separate_word_to_letters_diacritics(line)

        # initialize line data and diacritics lists and add start of sentence character
        x = [letters2id['<s>']]
        y = [diacritics2id['<s>']]
        for idx, char in enumerate(line):
            # skip character if it is only a dicritic
            if char in diacritics:
                continue
            # append character mapping to data list
            # if char not in letters:
            #     continue
            x.append(letters2id[char])
            
            # if character is not an arabic letter append whitespace to diacritics list
            if char not in letters:
                y.append(diacritics2id[''])
            # if character is an arabic letter append its discritics (following 1 or 2 characters) to diacritics list
            else:
                char_diac = ''
                if idx + 1 < len(line) and line[idx + 1] in diacritics:
                    char_diac = line[idx + 1]
                    if idx + 2 < len(line) and line[idx + 2] in diacritics and char_diac + line[idx + 2] in diacritics2id:
                        char_diac += line[idx + 2]
                    elif idx + 2 < len(line) and line[idx + 2] in diacritics and line[idx + 2] + char_diac in diacritics2id:
                        char_diac = line[idx + 2] + char_diac
                y.append(diacritics2id[char_diac])
        
        # append end of sentence character
        x.append(letters2id['</s>'])
        y.append(diacritics2id['</s>'])

        # convert diacritics integers to one_hot_encodings
        y = to_categorical(y, len(diacritics2id))

        # append line's data and diacritics lists to total data and diacritics lists
        X.append(x)
        Y.append(y)
    # convert lists to numpy arrays
    return X, Y
# print(map_data(["قَوْلُهُ"]))
# print(id2diacritics[0])
class DataGenerator(Sequence):
    ''' Costumized data generator to input line batches into the model '''
    def __init__(self, lines, batch_size,letters2id, diacritics2id):
        self.lines = lines
        self.batch_size = batch_size
        self.letters2id = letters2id
        self.diacritics2id = diacritics2id

    def __len__(self):
        return int(np.ceil(len(self.lines) / float(self.batch_size)))

    def __getitem__(self, idx):
        lines = self.lines[idx * self.batch_size:(idx + 1) * self.batch_size] # load number of sentences equal to batch size
        X_batch, Y_batch = map_data(lines) # map data to integers and one-hot-encodings
        X_max_seq_len = np.max([len(x) for x in X_batch])
        Y_max_seq_len = np.max([len(y) for y in Y_batch])
        assert(X_max_seq_len == Y_max_seq_len)
        X = list()
        for x in X_batch:
            x = list(x)
            x.extend([self.letters2id['<PAD>']] * (X_max_seq_len - len(x)))
            X.append(np.asarray(x))

        Y_tmp = list()
        for y in Y_batch:
            y_new = list(y)
            y_new.extend(to_categorical([self.diacritics2id['<PAD>']] * (Y_max_seq_len - len(y)), len(diacritics2id)))
            Y_tmp.append(np.asarray(y_new))
        Y_batch = Y_tmp
        X_batch = np.array(X)
        Y_batch = np.asarray(Y_batch)

        return (X_batch, Y_batch)

########################################################################################
def create_model(CHARACTERS_MAPPING, CLASSES_MAPPING ):
    ''' Creates diacritization model '''
    SelectedLSTM = LSTM # Bidirectional Long Short-Term Memory

    inputs = Input(shape=(None,))

    embeddings = Embedding(input_dim=len(CHARACTERS_MAPPING),
                           output_dim=25,
                           embeddings_initializer=glorot_normal(seed=961))(inputs)

    blstm1 = Bidirectional(SelectedLSTM(units=256,
                                     return_sequences=True,
                                     kernel_initializer=glorot_normal(seed=961)))(embeddings)
    dropout1 = Dropout(0.5)(blstm1)
    blstm2 = Bidirectional(SelectedLSTM(units=256,
                                     return_sequences=True,
                                     kernel_initializer=glorot_normal(seed=961)))(dropout1)
    dropout2 = Dropout(0.5)(blstm2)

    dense1 = TimeDistributed(Dense(units=512,
                                   activation='relu',
                                   kernel_initializer=glorot_normal(seed=961)))(dropout2)
    dense2 = TimeDistributed(Dense(units=512,
                                   activation='relu',
                                   kernel_initializer=glorot_normal(seed=961)))(dense1)

    output = TimeDistributed(Dense(units=len(CLASSES_MAPPING),
                                   activation='softmax',
                                   kernel_initializer=glorot_normal(seed=961)))(dense2)

    model = Model(inputs, output)

    # compile model
    # model.compile(loss='categorical_crossentropy', optimizer=Adam())
    # use this model to calc the accuracy
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

    return model
########################################################################################

def fit_model(model, epochs, batch_size, train_split,val_split,letters2id, diacritics2id):
    ''' Fits model '''


    # create training and validation generators
    training_generator = DataGenerator(train_split, batch_size,letters2id, diacritics2id)
    val_generator = DataGenerator(val_split, batch_size,letters2id, diacritics2id)

    # fit model
    history = model.fit(x=training_generator,
              validation_data=val_generator,
              epochs=epochs
              )

    # return history
    return history
########################################################################################
def remove_diacritics(data_raw):
  ''' Returns undiacritized text'''
  return data_raw.translate(str.maketrans('', '', ''.join(diacritics)))
def predict(line, model):
    ''' predict test line '''
    X, _ = map_data([line])
    # for x in X[0]:
    #   print(id2letters[x])
    predictions = model.predict(X,verbose=0).squeeze()
    # print(len(remove_diacritics(line)),len(predictions))

    # print(predictions[0])
    # # get most probable diacritizations for each character
    predictions = predictions[1:]
    # initialize empty output line
    output = ''
    char_list=[]
    diac_list=[]
    # loop on input characters and predicted diacritizations
    for char, prediction in zip(remove_diacritics(line), predictions):
        if char != ' ': # ignore spaces
            char_list.append(char)
            diac_list.append(np.argmax(prediction))
    # ind=0
    # for prediction in predictions:
    #     char=X[ind]
        # print(char)
        # # ind+=1
        # print(char,id2diacritics[np.argmax(prediction)])
        # print(np.argmax(prediction),len(prediction),len(id2diacritics))

        # append character
        output += char
        # if character is not an arabic letter continue
        if char not in letters:
            continue

        if '<' in id2diacritics[np.argmax(prediction)]:
            continue

        # if character in arabic letters append predicted diacritization
        output += id2diacritics[np.argmax(prediction)]

    return output,char_list,diac_list

print(id2diacritics[0])

NameError: name 'Sequence' is not defined

### Read Data

In [ ]:
training_dataset = read_training_dataset()
dev_dataset = read_dev_dataset()
combined_dataset = training_dataset 

data, labels, cleaned_corpus = extract_sentences(combined_dataset)
val_data, val_labels, val_cleaned_corpus = extract_sentences(dev_dataset)


### Pre-process and clean data

In [ ]:
# 1- Clean the data

# write the clean corpora to file
# with open("./output_data/cleaned_corpus.txt", "w", encoding="utf-8") as f:
#     for l in cleaned_corpus:
#         f.write(l + "\n")

# with open("./output_data/training_data.txt", "w", encoding="utf-8") as f:
#     for d in data:
#         f.write(str(d) + "\n")

# with open("./output_data/labeled/training_labels.txt", "w", encoding="utf-8") as f:
#     for l in labels:
#         f.write(str(l) + "\n")


In [ ]:
# 2- Tokenize to vocab and words
# vocab = set()
# tokenized_data = []
# vocab, tokenized_word_sentence, tokenized_letter_sentence, tokenized_diacritics_sentence = tokenize_to_vocab(labels, vocab)
# stemmedVocab = []

# Save the vocab to file each word in a line
# with open("./output_data/vocab.txt", "w", encoding="utf-8") as f:
#     for v in vocab:
#         f.write(str(v) + "\n")

# # save the tokenized data sentence, as a form of list with <s> in index 0 and </s> in the last index
#         # in between are words
# with open("./output_data/tokenized_data_sentences.txt", "w", encoding="utf-8") as f:
#     for ts in tokenized_word_sentence:
#         f.write(str(ts) + "\n")

# # save the tokenized data sentence, as a form of list with <s> in index 0 and </s> in the last index
#         # in between are lists of letters
# with open("./output_data/tokenized_data_letters.txt", "w", encoding="utf-8") as f:
#     for tl in tokenized_letter_sentence:
#         f.write(str(tl) + "\n")

# # save the tokenized data sentence, as a form of list with <s> in index 0 and </s> in the last index
#         # in between are lists of diacritics
# with open("./output_data/tokenized_data_diacritics.txt", "w", encoding="utf-8") as f:
#     for td in tokenized_diacritics_sentence:
#         f.write(str(td) + "\n")


In [28]:

#1- We then pass the embeddings to an RNN Model
# This model consists of an embedding layer, an RNN layer, and a fully connected layer. 
# The embedding layer transforms the input words (represented as integers) into dense vectors of fixed size. 
# The RNN layer processes these word embeddings sequentially, 
# maintaining an internal state that encodes information about the sequence so far. T
# he fully connected layer transforms the output of the RNN layer to the desired output size.
# with open( './constants/RNN_BIG_CHARACTERS_MAPPING.pickle', 'rb') as file:
#     CHARACTERS_MAPPING = pkl.load(file)
model = create_model(letters2id, diacritics2id)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 25)          975       
                                                                 
 bidirectional_4 (Bidirecti  (None, None, 512)         577536    
 onal)                                                           
                                                                 
 dropout_4 (Dropout)         (None, None, 512)         0         
                                                                 
 bidirectional_5 (Bidirecti  (None, None, 512)         1574912   
 onal)                                                           
                                                                 
 dropout_5 (Dropout)         (None, None, 512)         0   

### FITING 


In [ ]:
start_time = time.time()
hist_1 = fit_model(model, 10, 128, labels, val_labels,letters2id, diacritics2id)
end_time = time.time()
print('--- %s seconds ---' % round(end_time - start_time, 2))


In [ ]:
# plot loss history
plt.plot(hist_1.history['loss'])
plt.plot(hist_1.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

In [ ]:
# print(predict("العصفور فوق الشجرة", model))
test_data = read_dev_dataset('./dataset/test.txt')
test_data,test_labels,test_corpus = extract_sentences(test_data)
data_gen = DataGenerator(test_labels, 1,letters2id, diacritics2id)
acc = model.evaluate(data_gen)[1]
print("accracy in the test set is : ",acc)

In [ ]:
print(predict('ذهب زياد الي المدرسة', model)[0])
print(predict("العصفور فوق الشجرة الكبيرة", model)[0])
print(predict("احمد يحب حنان", model)[0])
print(predict("الولد يلعب تحت الشجره", model)[0])

### SAVE

In [ ]:
model.save('model_epoch1')

### Load model

In [ ]:
from tensorflow.keras.models import load_model
new_model = load_model('model_epoch1')

In [ ]:
print(predict('ذهب زياد الي المدرسة', model)[0])


### load test data and clean it

In [31]:
import csv
import time
# test_data = read_dev_dataset('./dataset/competition/test_no_diacritics.txt')
test_data = read_dev_dataset('./test.txt')
data, labels, cleaned_corpus = extract_sentences(test_data)

last_id=0
# open csv fiel as write
file = open('result.csv', 'w', newline='', encoding='utf-8')
writer = csv.writer(file)
writer.writerow(['ID', 'label'])

print(data[0])
data = [" ".join(data) ]
print(len(data[0]))
start_time = time.time()
for i in range(len(data)):
    out , char_list, diac_list = predict(data[i], model)
    for j in range(len(char_list)):
        if char_list[j] == ' ':
            continue
        # out_list.append([last_id, diac_list[j]])
        writer.writerow([last_id, diac_list[j]])
        last_id+=1

file.close()

end_time = time.time()
print('--- %s seconds ---' % round(end_time - start_time, 2))
print(last_id)


1
ليس للوكيل بالقبض أن يبرأ المدين أو يهب الدين له أو يأخذ رهنا من المدين في مقابل الدين أو يقبل إحالته على شخص آخر لكن له أن يأخذ كفيلا لكن ليس له أن يأخذ كفيلا بشرط براءة الأصيل انظر المادة
226
--- 1.9 seconds ---
182


In [ ]:
print(last_id)

### CALC THE ACCURACY IF WE HAVE THE GOLD OUTPUT

In [32]:
import pandas as pd

# Load the datasets
gold_data = pd.read_csv('sample_test_set_gold.csv')
predicted_data = pd.read_csv('result.csv')  # Assuming this contains your predictions

# Ensure both dataframes have the same length
assert len(gold_data) == len(predicted_data), "Datasets must have the same number of rows"

# Compare and calculate accuracy
correct_predictions = 0
for gold_label, predicted_label in zip(gold_data['label'], predicted_data['label']):
    if gold_label == predicted_label:
        correct_predictions += 1

accuracy = correct_predictions / (len(gold_data)-1)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.04


In [7]:
import pandas as pd

# Read the data
gold_data = pd.read_csv('./res/last_char_submission1.csv')
predicted_data = pd.read_csv('./res/submission_7.csv')

# Create a dictionary from gold_data for quick lookup
gold_dict = dict(zip(gold_data['ID'], gold_data['label']))

# Update predicted_data with labels from gold_data where IDs match
for index, row in predicted_data.iterrows():
    id = row['ID']
    if id in gold_dict:
        predicted_data.at[index, 'label'] = gold_dict[id]

# Write the updated data to a new file
predicted_data.to_csv('./res/updated_submission.csv', index=False)


In [9]:
import pandas as pd

# Read the data
gold_data = pd.read_csv('./res/submission.csv')[:1000]
predicted_data = pd.read_csv('./res/submission_7.csv')[:1000]

miss=0
correct_predictions=0
ind=0
for gold_label, predicted_label in zip(gold_data['label'], predicted_data['label']):
    if gold_label == predicted_label:
        correct_predictions += 1
    else :
        print(ind,gold_label,predicted_label)
        miss+=1
    ind+=1
print(miss)


3 0 4
4 3 6
6 6 4
7 2 14
8 2 4
9 0 4
10 0 14
11 0 6
13 8 6
14 2 4
15 4 0
17 2 0
18 2 6
19 4 0
20 6 0
21 0 14
22 0 6
24 0 4
25 4 14
26 14 0
27 1 0
28 14 6
29 2 0
30 8 0
32 4 14
33 0 14
34 0 8
35 14 6
36 2 0
38 12 2
39 2 0
40 0 6
41 8 0
42 2 6
43 0 2
44 6 0
45 3 0
46 0 6
47 6 1
48 2 14
49 0 4
51 0 14
52 0 6
53 4 0
54 14 4
55 4 14
56 0 4
57 14 4
58 0 14
59 4 2
61 6 14
62 0 4
64 6 14
65 4 14
66 0 8
67 4 6
68 13 4
69 14 0
71 0 2
72 0 6
73 14 0
74 4 2
75 0 14
82 2 0
83 3 14
85 0 6
86 0 5
87 6 14
88 4 0
90 4 0
91 14 4
92 5 6
94 14 2
97 4 0
98 14 6
99 0 2
100 6 0
102 6 4
103 3 14
104 4 1
105 0 14
107 0 4
108 14 6
110 0 6
111 6 0
113 0 2
114 8 0
115 14 6
116 4 0
117 0 6
118 14 2
120 8 0
121 0 4
122 2 14
123 0 1
124 0 14
125 6 4
126 2 0
127 0 6
128 0 4
129 14 0
131 0 14
133 14 4
134 0 14
136 14 0
137 6 4
138 5 14
139 0 4
140 0 2
141 0 6
142 0 2
143 2 6
144 0 14
147 2 14
148 6 8
149 2 0
153 14 6
154 4 0
155 4 6
156 6 4
157 4 10
159 6 14
160 2 8
161 8 6
162 4 0
163 4 14
164 0 4
165 0 8
166 2 0
167